# 6. 크롤링을 통한 일별시세 데이터 수집

- 네이버 금융 개별종목 수집

<br>

## 6.1 네이버 증권의 개별종목 일별 시세 데이터 가져오는 방법

### 6.1.1 라이브러리 로드

In [20]:
import pandas as pd
import requests as rq

<br>

### 6.1.2 수집할 URL 정하기

In [2]:
# 종목 URL 만들기
url = 'https://finance.naver.com/item/sise.nhn?code=005930'

<br>

### 6.1.3 pandas를 통해 데이터 수집하기

- `pd.read_html()`을 이용하여 url의 page내의 값을 DataFrame으로 받아온다.
- `cp949`는 한글 인코딩을 위해 사용한다.
- 기본 인코딩 설정은 `utf-8`이며, 네이버의 일별 시세는 `cp949` 인코딩으로 불러올 수 있다.
- 데이터를 로드했을 때 한글 인코딩이 깨진다면 대부분 `cp949`로 불러올 수 있다.

In [3]:
table = pd.read_html(url, encoding='cp949')

In [4]:
len(table)

12

In [5]:
table[0]

,0,1,2
0,"전일 85,600 85,600","고가 85,60085,600 (상한가 111,000111,000 )","거래량 31,598,976 31,598,976"
1,"시가 83,20083,200","저가 83,20083,200 (하한가 60,000 )","거래대금 2,654,762 2,654,762 백만"


In [6]:
table[1]

,0,1,2,3
0,현재가,83700,매도호가,83800
1,전일대비,"하락 1,900",매수호가,83700
2,등락률(%),-2.22%,전일가,85600
3,거래량,31598976,시가,83200
4,거래대금(백만),2654762,고가,85600
5,액면가,100원,저가,83200
6,NaN,NaN,NaN,NaN
7,상한가,111000,전일상한,112500
8,하한가,60000,전일하한,60700
9,PER,22.86,EPS,3662


In [7]:
table[2]

,매도잔량,매도호가,Unnamed: 2,매수호가,매수잔량
0,NaN,NaN,NaN,NaN,NaN
1,19527.0,84200.0,NaN,NaN,NaN
2,16359.0,84100.0,NaN,NaN,NaN
3,81628.0,84000.0,NaN,NaN,NaN
4,121499.0,83900.0,NaN,NaN,NaN
5,132348.0,83800.0,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,83700.0,51142.0


In [8]:
table[3]

,0,1,2
0,535945,잔량합계,3133286


In [9]:
table[4]

,0,1
0,시가총액,"499조 6,708 억원"
1,시가총액순위,코스피 1위
2,상장주식수,5969782550
3,액면가l매매단위,100원 l 1주


In [10]:
table[5]

,0,1
0,외국인한도주식수(A),5969782550
1,외국인보유주식수(B),3299185995
2,외국인소진율(B/A),55.26%


In [11]:
table[6]

,0,1
0,투자의견l목표주가,"4.00매수 l 102,040"
1,52주최고l최저,"96,800 l 42,300"


In [12]:
table[7]

,0,1
0,PERlEPS(2020.09),"22.86배 l 3,662원"
1,추정PERlEPS,"16.00배 l 5,189원"
2,PBRlBPS (2020.09),"∞배 l 39,446원"
3,배당수익률,NaN


<br>

### 6.1.4 원하는 데이터를 가져오지 못한 이유

- 시간별 시세와 일별 시세 데이터가 수집되지 않았다.  
  
  
- 크롬 개발자 도구 > Network 에서 해당 페이지를 로드할 때 다운로드 되는 파일들을 볼 수 있다.
  - `XHR` : 비동기 통신으로 가져오는 파일들을 보여줌
  - `Doc` : 문서 파일들을 보여줌  
  
  
- 이번 경우에는 `Doc` 에 표출되는 파일 중 가져오고자 하는 일별 시세 정보가 들어 있는 것을 볼 수 있다.  
(페이지 번호를 눌렀을 때 해당 파일이 새로 추가되는 것 확인 가능)
- 이를 통해 일별 시세 데이터를 수집할 수 있는 url을 얻을 수 있다.

<br>

### 6.1.5 일별 시세 데이터 수집

> 기존 방법이 네이버 사이트에서 제한되서 아래 코드로 수집 가능

In [26]:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
url = 'https://finance.naver.com/item/sise_day.nhn?code=005930&page=2'
r = rq.get(url, headers={"user-agent": user_agent})
tb = r.content.decode('euc-kr')
table = pd.read_html(tb)
len(table)

2

In [27]:
temp = table[0]
temp

,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021.01.14,89700.0,0.0,88700.0,90000.0,88700.0,26393970.0
2,2021.01.13,89700.0,900.0,89800.0,91200.0,89100.0,36068848.0
3,2021.01.12,90600.0,400.0,90300.0,91400.0,87800.0,48682416.0
4,2021.01.11,91000.0,2200.0,90000.0,96800.0,89500.0,90306177.0
5,2021.01.08,88800.0,5900.0,83300.0,90000.0,83000.0,59013307.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021.01.07,82900.0,700.0,82800.0,84200.0,82700.0,32644642.0


<br>

### 6.1.6 결측치 제거

In [29]:
# axis=1 : 행 기준
# how='all' : 해당 행의 모든 데이터가 결측치 일때만 제거
temp.dropna(axis=0, how='all')

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.01.14,89700.0,0.0,88700.0,90000.0,88700.0,26393970.0
2,2021.01.13,89700.0,900.0,89800.0,91200.0,89100.0,36068848.0
3,2021.01.12,90600.0,400.0,90300.0,91400.0,87800.0,48682416.0
4,2021.01.11,91000.0,2200.0,90000.0,96800.0,89500.0,90306177.0
5,2021.01.08,88800.0,5900.0,83300.0,90000.0,83000.0,59013307.0
9,2021.01.07,82900.0,700.0,82800.0,84200.0,82700.0,32644642.0
10,2021.01.06,82200.0,1700.0,83300.0,84500.0,82100.0,42089013.0
11,2021.01.05,83900.0,900.0,81600.0,83900.0,81600.0,35335669.0
12,2021.01.04,83000.0,2000.0,81000.0,84400.0,80200.0,38655276.0
13,2020.12.30,81000.0,2700.0,77400.0,81300.0,77300.0,29417421.0


<br>

### 6.1.7 종목코드 변수로 넘겨주기

In [33]:
item_code = '326030'
item_name = 'SK바이오팜'

url = f'https://finance.naver.com/item/sise_day.nhn?code={item_code}&page=2'
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"

r = rq.get(url, headers={"user-agent": user_agent})
tb = r.content.decode('euc-kr')

table = pd.read_html(tb)
table[0].dropna(axis=0, how='all')

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.01.14,155000.0,2000.0,153500.0,156500.0,152000.0,286485.0
2,2021.01.13,153000.0,1500.0,154500.0,154500.0,152000.0,262693.0
3,2021.01.12,154500.0,1500.0,155500.0,156000.0,152500.0,400078.0
4,2021.01.11,156000.0,3500.0,159000.0,159500.0,154500.0,610221.0
5,2021.01.08,159500.0,1000.0,159500.0,161000.0,155000.0,703920.0
9,2021.01.07,158500.0,1000.0,160000.0,160500.0,157500.0,486353.0
10,2021.01.06,159500.0,1000.0,161500.0,162500.0,158000.0,626403.0
11,2021.01.05,160500.0,6000.0,155000.0,162500.0,152500.0,1377082.0
12,2021.01.04,154500.0,14500.0,149500.0,157000.0,149000.0,3338448.0
13,2020.12.30,169000.0,1000.0,168000.0,169500.0,167000.0,221674.0


In [34]:
item_code = '352820'
item_name = '빅히트'

url = f'https://finance.naver.com/item/sise_day.nhn?code={item_code}&page=2'
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"

r = rq.get(url, headers={"user-agent": user_agent})
tb = r.content.decode('euc-kr')

table = pd.read_html(tb)
table[0].dropna(axis=0, how='all')

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.01.14,166000.0,2500.0,166500.0,170500.0,163500.0,341487.0
2,2021.01.13,168500.0,15000.0,153500.0,180000.0,153000.0,2079262.0
3,2021.01.12,153500.0,1000.0,154500.0,157000.0,150000.0,202504.0
4,2021.01.11,154500.0,6000.0,162000.0,162500.0,154000.0,253932.0
5,2021.01.08,160500.0,3500.0,158000.0,162000.0,155000.0,285332.0
9,2021.01.07,157000.0,1500.0,158500.0,159500.0,157000.0,139860.0
10,2021.01.06,158500.0,0.0,160000.0,161000.0,157500.0,148194.0
11,2021.01.05,158500.0,500.0,157000.0,159000.0,155500.0,168962.0
12,2021.01.04,158000.0,2000.0,160000.0,160500.0,156000.0,152926.0
13,2020.12.30,160000.0,1500.0,158500.0,161000.0,157500.0,117495.0


<br>

## 6.2 페이지별 데이터 수집 함수 구현

In [42]:
def get_day_list(item_code, page_no):
    """
    일자별 시세를 페이지별로 수집
    """
    
    url = f'https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}'
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"

    r = rq.get(url, headers={"user-agent": user_agent})
    tb = r.content.decode('euc-kr')

    table = pd.read_html(tb)
    df_day = table[0].dropna(axis=0, how='all')
    
    return df_day

In [43]:
get_day_list('005930', '1') # 삼성전자

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.01.28,83700.0,1900.0,83200.0,85600.0,83200.0,31598976.0
2,2021.01.27,85600.0,1100.0,86600.0,87700.0,85600.0,26423070.0
3,2021.01.26,86700.0,2700.0,88800.0,89200.0,86500.0,33178936.0
4,2021.01.25,89400.0,2600.0,87000.0,89900.0,86300.0,27258534.0
5,2021.01.22,86800.0,1300.0,89000.0,89700.0,86800.0,30861661.0
9,2021.01.21,88100.0,900.0,87500.0,88600.0,86500.0,25318011.0
10,2021.01.20,87200.0,200.0,89000.0,89000.0,86500.0,25211127.0
11,2021.01.19,87000.0,2000.0,84500.0,88000.0,83600.0,39895044.0
12,2021.01.18,85000.0,3000.0,86600.0,87300.0,84100.0,43227951.0
13,2021.01.15,88000.0,1700.0,89800.0,91800.0,88000.0,33431809.0


<br>

## 6.3 반복문을 통한 전체 일자 데이터 수집

> (주의) 기간이 긴 데이터를 수집할 때는 서버에 부담을 주지 않기 위해 `time.sleep()` 를 지정해줘야 한다.

In [64]:
import time

item_code = '326030'
item_name = 'SK바이오팜'
page_no = 1
item_list = []

while True:
    df_day = get_day_list(item_code, page_no)
    item_list.append(df_day)
    page_no = page_no + 1
    
    time.sleep(0.1)
    
    if len(df_day) < 10:
        break

In [65]:
len(item_list)

15

In [66]:
item_list

[            날짜        종가     전일비        시가        고가        저가       거래량
 1   2021.01.28  148000.0  1500.0  148000.0  150500.0  147000.0  197915.0
 2   2021.01.27  149500.0  2500.0  152000.0  153000.0  149500.0  167932.0
 3   2021.01.26  152000.0     0.0  153000.0  155000.0  151500.0  245969.0
 4   2021.01.25  152000.0  1500.0  151000.0  153500.0  150500.0  250843.0
 5   2021.01.22  150500.0  1000.0  152500.0  153000.0  149000.0  211286.0
 9   2021.01.21  151500.0  3500.0  148000.0  156500.0  147500.0  641065.0
 10  2021.01.20  148000.0     0.0  147500.0  148000.0  146000.0  243971.0
 11  2021.01.19  148000.0     0.0  146500.0  148000.0  145000.0  274255.0
 12  2021.01.18  148000.0  5000.0  151500.0  151500.0  147000.0  419631.0
 13  2021.01.15  153000.0  2000.0  156000.0  156500.0  152500.0  253543.0,
             날짜        종가      전일비        시가        고가        저가        거래량
 1   2021.01.14  155000.0   2000.0  153500.0  156500.0  152000.0   286485.0
 2   2021.01.13  153000.0   1500.

<br>

## 6.4 수집한 데이터를 하나의 데이터프레임으로 합치기

In [67]:
df = pd.concat(item_list)
df

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.01.28,148000.0,1500.0,148000.0,150500.0,147000.0,197915.0
2,2021.01.27,149500.0,2500.0,152000.0,153000.0,149500.0,167932.0
3,2021.01.26,152000.0,0.0,153000.0,155000.0,151500.0,245969.0
4,2021.01.25,152000.0,1500.0,151000.0,153500.0,150500.0,250843.0
5,2021.01.22,150500.0,1000.0,152500.0,153000.0,149000.0,211286.0
...,...,...,...,...,...,...,...
12,2020.07.08,217000.0,500.0,214500.0,225000.0,198000.0,5990009.0
13,2020.07.07,216500.0,2000.0,219000.0,269500.0,210000.0,10105187.0
1,2020.07.06,214500.0,49500.0,214500.0,214500.0,193500.0,7126211.0
2,2020.07.03,165000.0,38000.0,165000.0,165000.0,165000.0,711921.0


In [68]:
len(df)

143

In [69]:
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2021.01.28,148000.0,1500.0,148000.0,150500.0,147000.0,197915.0
2,2021.01.27,149500.0,2500.0,152000.0,153000.0,149500.0,167932.0
3,2021.01.26,152000.0,0.0,153000.0,155000.0,151500.0,245969.0
4,2021.01.25,152000.0,1500.0,151000.0,153500.0,150500.0,250843.0
5,2021.01.22,150500.0,1000.0,152500.0,153000.0,149000.0,211286.0


In [70]:
df.tail()

,날짜,종가,전일비,시가,고가,저가,거래량
12,2020.07.08,217000.0,500.0,214500.0,225000.0,198000.0,5990009.0
13,2020.07.07,216500.0,2000.0,219000.0,269500.0,210000.0,10105187.0
1,2020.07.06,214500.0,49500.0,214500.0,214500.0,193500.0,7126211.0
2,2020.07.03,165000.0,38000.0,165000.0,165000.0,165000.0,711921.0
3,2020.07.02,127000.0,29000.0,98000.0,127000.0,98000.0,698642.0


<br>

## 6.5 데이터 전처리

### 6.5.1 파생 변수 만들기 (종목코드, 종목명)

In [71]:
df['종목코드'] = item_code
df['종목명'] = item_name

df.head()

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드,종목명
1,2021.01.28,148000.0,1500.0,148000.0,150500.0,147000.0,197915.0,326030,SK바이오팜
2,2021.01.27,149500.0,2500.0,152000.0,153000.0,149500.0,167932.0,326030,SK바이오팜
3,2021.01.26,152000.0,0.0,153000.0,155000.0,151500.0,245969.0,326030,SK바이오팜
4,2021.01.25,152000.0,1500.0,151000.0,153500.0,150500.0,250843.0,326030,SK바이오팜
5,2021.01.22,150500.0,1000.0,152500.0,153000.0,149000.0,211286.0,326030,SK바이오팜


<br>

### 6.5.2 컬럼 순서 변경

In [73]:
df.columns

Index(['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량', '종목코드', '종목명'], dtype='object')

- `copy()` 함수를 사용하면 깊은 복사를 하게 된다.

In [75]:
cols = [ '종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
df = df[cols].copy()
df.head()

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,326030,SK바이오팜,2021.01.28,148000.0,1500.0,148000.0,150500.0,147000.0,197915.0
2,326030,SK바이오팜,2021.01.27,149500.0,2500.0,152000.0,153000.0,149500.0,167932.0
3,326030,SK바이오팜,2021.01.26,152000.0,0.0,153000.0,155000.0,151500.0,245969.0
4,326030,SK바이오팜,2021.01.25,152000.0,1500.0,151000.0,153500.0,150500.0,250843.0
5,326030,SK바이오팜,2021.01.22,150500.0,1000.0,152500.0,153000.0,149000.0,211286.0


<br>

### 6.5.3 중복된 데이터 제거

- `drop_duplicates()`

In [77]:
print(df.shape)
df = df.drop_duplicates()
print(df.shape)

(143, 9)
(143, 9)


<br>

## 6.6 기술통계값 구하기

In [78]:
df.describe()

,종가,전일비,시가,고가,저가,거래량
count,143.000000,143.000000,143.000000,143.000000,143.000000,1.430000e+02
mean,170265.734266,3937.062937,170723.776224,174055.944056,167090.909091,6.811557e+05
std,15532.769109,6139.191559,16812.729743,18312.920177,15335.868428,1.202942e+06
min,127000.000000,0.000000,98000.000000,127000.000000,98000.000000,1.074820e+05
25%,158750.000000,1000.000000,159500.000000,161250.000000,155000.000000,2.106470e+05
50%,170000.000000,2500.000000,170500.000000,172000.000000,168500.000000,3.215860e+05
75%,179250.000000,4500.000000,179500.000000,182750.000000,176250.000000,6.337340e+05
max,217000.000000,49500.000000,219000.000000,269500.000000,210000.000000,1.010519e+07


<br>

### 6.6.1 과학적 기수법 (E notation)

In [80]:
df['거래량'].describe()

count    1.430000e+02
mean     6.811557e+05
std      1.202942e+06
min      1.074820e+05
25%      2.106470e+05
50%      3.215860e+05
75%      6.337340e+05
max      1.010519e+07
Name: 거래량, dtype: float64

In [81]:
1.430000e+02

143.0

In [82]:
1.43 * (10 ** 2)

143.0

In [83]:
1.010519e+07

10105190.0

<br>

## 6.7 파일 저장

### 6.7.1 최근 날짜 구해서 파일명 만들기

In [85]:
date = df.iloc[0]['날짜']
date

'2021.01.28'

In [89]:
file_name = f'{item_name}_{item_code}_{date}.csv'
file_name

'SK바이오팜_326030_2021.01.28.csv'

<br>

### 6.7.2 csv 파일로 저장

In [90]:
df.to_csv(file_name, index=False)

In [91]:
# 제대로 저장되었는 지 확인
pd.read_csv(file_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,326030,SK바이오팜,2021.01.28,148000.0,1500.0,148000.0,150500.0,147000.0,197915.0
1,326030,SK바이오팜,2021.01.27,149500.0,2500.0,152000.0,153000.0,149500.0,167932.0
2,326030,SK바이오팜,2021.01.26,152000.0,0.0,153000.0,155000.0,151500.0,245969.0
3,326030,SK바이오팜,2021.01.25,152000.0,1500.0,151000.0,153500.0,150500.0,250843.0
4,326030,SK바이오팜,2021.01.22,150500.0,1000.0,152500.0,153000.0,149000.0,211286.0
...,...,...,...,...,...,...,...,...,...
138,326030,SK바이오팜,2020.07.08,217000.0,500.0,214500.0,225000.0,198000.0,5990009.0
139,326030,SK바이오팜,2020.07.07,216500.0,2000.0,219000.0,269500.0,210000.0,10105187.0
140,326030,SK바이오팜,2020.07.06,214500.0,49500.0,214500.0,214500.0,193500.0,7126211.0
141,326030,SK바이오팜,2020.07.03,165000.0,38000.0,165000.0,165000.0,165000.0,711921.0
